In [1]:
%load_ext autoreload
%autoreload 2
notebook_fixed_dir = False

In [2]:
# this cell can only be called once
import os
if not notebook_fixed_dir:
    os.chdir('..')
    notebook_fixed_dir = True
print(os.getcwd())

/home/svcl-oowl/brandon/research/sil_consistent_at_inference


In [16]:
import pprint
import glob
import os

import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image


In [17]:
IMAGE_DATASET_PATH = 'data/test'
#IMAGE_DATASET_PATH = 'data/pix3d_transparent_224/chair'
new_image_size = 224


In [18]:
# convert images to make all white pixels transparent
# based off https://stackoverflow.com/questions/765736/using-pil-to-make-all-white-pixels-transparent

for root, dirs, files in os.walk(os.path.join(IMAGE_DATASET_PATH)):
    for f in files:
        
        image_path = os.path.join(root,f)
        img = Image.open(image_path)
        img = img.convert("RGBA")
        datas = img.getdata()

        newData = []
        for item in datas:
            if item[0] >= 255 and item[1] >= 255 and item[2] >= 255:
                newData.append((255, 255, 255, 0))
            else:
                newData.append(item)

        img.putdata(newData)
        new_img_path = image_path.split('.')
        new_img_path[-1] = 'png'
        new_img_path = '.'.join(new_img_path)
        img.save(new_img_path, "PNG")

In [15]:
# resize images
for root, dirs, files in os.walk(os.path.join(IMAGE_DATASET_PATH)):
    for f in files:
        image_path = os.path.join(root,f)
        image = Image.open(image_path).resize((new_image_size,new_image_size), resample = PIL.Image.BILINEAR)
        image.save(image_path)

In [20]:
im = cv2.imread("data/test/df7735e2bce09a511f98c0761af40e04.jpg")
h,w,chn = im.shape
seed = (1,1)
#seed = (w/2,h/2)

mask = np.zeros((h+2,w+2),np.uint8)

floodflags = 4
floodflags |= cv2.FLOODFILL_MASK_ONLY
floodflags |= (255 << 8)

num,im,mask,rect = cv2.floodFill(im, mask, seed, (255,0,0), (10,)*3, (10,)*3, floodflags)

cv2.imwrite("data/test/flood.png", mask)

True

In [ ]:
# do binary/adaptive thresholding
# find contours
# choose largest contour and fill in